# Steganography Demo Notebook

This notebook demonstrates how to use the steganography project to hide and extract data in different types of files.

## Setup

First, let's import the necessary modules and set up the environment.

In [2]:
import os
import sys

# Add the parent directory to the path to import project modules
sys.path.append(os.path.abspath('..'))

from steganography.image_stego import LSBImageSteganography
from steganography.audio_stego import LSBAudioSteganography
from steganography.text_stego import WhitespaceTextSteganography, ZeroWidthTextSteganography
from core.file_utils import read_binary_file, write_binary_file, read_text_file, write_text_file
from utils.bit_utils import bytes_to_bits, bits_to_bytes
from core.huffman import huffman_encode, huffman_decode

# Set up paths for example files
EXAMPLES_DIR = os.path.abspath('.')
print(f"Examples directory: {EXAMPLES_DIR}")

Examples directory: /home/tsitonyavo/Documents/Studies/MrTsinjo/truc2/steganography_project/examples


## 1. Image Steganography

Let's start by hiding data in an image file using the LSB (Least Significant Bit) method.

In [ ]:
# Load an example image
from PIL import Image
import matplotlib.pyplot as plt

# You should replace this with your own image path
image_path = os.path.join(EXAMPLES_DIR, 'input_image.png')

# If the example image doesn't exist, create a sample one
if not os.path.exists(image_path):
    # Create a simple colored image
    img = Image.new('RGB', (300, 200), color=(73, 109, 137))
    img.save(image_path)
    print(f"Created sample image at {image_path}")

# Display the image
img = Image.open(image_path)
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title("Original Image")
plt.axis('off')
plt.show()

print(f"Image dimensions: {img.size}")

### Hide Data in the Image

Now let's hide some text data in this image.

In [ ]:
# Create some secret text
secret_text = "This is a secret message that will be hidden inside the image!"
secret_data = secret_text.encode('utf-8')

# Initialize the LSB image steganography object
# You can provide a password for added security
password = "mysecretpassword"  # Optional
image_stego = LSBImageSteganography(password)

# Hide the data in the image
output_image_path = os.path.join(EXAMPLES_DIR, 'output_image.png')
result_path = image_stego.hide_data(image_path, secret_data, output_image_path)

print(f"Data hidden in image: {result_path}")

# Display the output image
output_img = Image.open(output_image_path)
plt.figure(figsize=(8, 6))
plt.imshow(output_img)
plt.title("Image with Hidden Data")
plt.axis('off')
plt.show()

### Extract Data from the Image

Now let's extract the hidden data from the output image.

In [ ]:
# Extract the hidden data
# Make sure to use the same password if one was used during hiding
extracted_data = image_stego.extract_data(output_image_path)

# Convert the extracted bytes back to text
extracted_text = extracted_data.decode('utf-8')

print(f"Extracted text: {extracted_text}")

# Verify the extraction was successful
if extracted_text == secret_text:
    print("✅ Extraction successful! The extracted text matches the original.")
else:
    print("❌ Extraction failed! The texts don't match.")

## 2. Text Steganography

Now let's try hiding data in a text file using whitespace steganography.

In [3]:
# Load the example text file
text_path = os.path.join(EXAMPLES_DIR, 'input_text.txt')

# Display the text content
try:
    with open(text_path, 'r', encoding='utf-8') as f:
        text_content = f.read()
    
    print("Original Text Content:")
    print("---------------------")
    print(text_content)
    print("---------------------")
except FileNotFoundError:
    print(f"Example text file not found at {text_path}")
    # Create a sample text file
    sample_text = """This is a sample text file for steganography.
It contains multiple lines to demonstrate how data
can be hidden using whitespace or zero-width characters.
The recipient of this file would never suspect
that it contains hidden information."""
    
    with open(text_path, 'w', encoding='utf-8') as f:
        f.write(sample_text)
    
    print(f"Created sample text file at {text_path}")
    print("Content:")
    print("---------------------")
    print(sample_text)
    print("---------------------")

Original Text Content:
---------------------
This is an example text file that will be used for steganography testing.
It contains multiple lines of text to demonstrate how data can be hidden
in text files using different steganography techniques.

The steganography module supports two main methods for text files:
1. Whitespace steganography - using spaces and tabs at the end of lines
2. Zero-width character steganography - using invisible Unicode characters

Both methods allow hiding binary data without visibly altering the appearance
of the text file to human readers.

---------------------


### Hide Data in the Text

Let's hide some binary data in the text file using whitespace steganography.

In [6]:
# Create some secret data (a short image in this case)
from PIL import Image, ImageDraw
import io

# Create a simple image
secret_img = Image.new('RGB', (50, 50), color=(255, 255, 255))
draw = ImageDraw.Draw(secret_img)
draw.rectangle([(10, 10), (40, 40)], fill=(255, 0, 0))

# Save to bytes
img_bytes = io.BytesIO()
secret_img.save(img_bytes, format='PNG')
secret_data = img_bytes.getvalue()

print(f"Size of secret data: {len(secret_data)} bytes")

# Initialize the whitespace text steganography object
text_stego = WhitespaceTextSteganography()

# Hide the data in the text file
output_text_path = os.path.join(EXAMPLES_DIR, 'output_text.txt')
result_path = text_stego.hide_data(text_path, secret_data, output_text_path)

print(f"Data hidden in text: {result_path}")

# Display the output text (it should look identical to the original)
with open(output_text_path, 'r', encoding='utf-8') as f:
    stego_text = f.read()

print("\nText with Hidden Data (visually identical to the original):")
print("---------------------")
print(stego_text)
print("---------------------")

Size of secret data: 164 bytes
Data hidden in text: /home/tsitonyavo/Documents/Studies/MrTsinjo/truc2/steganography_project/examples/output_text.txt

Text with Hidden Data (visually identical to the original):
---------------------
This is an example text file that will be used for steganography testing. 
It contains multiple lines of text to demonstrate how data can be hidden 
in text files using different steganography techniques. 
 
The steganography module supports two main methods for text files: 
1. Whitespace steganography - using spaces and tabs at the end of lines 
2. Zero-width character steganography - using invisible Unicode characters 
 
Both methods allow hiding binary data without visibly altering the appearance 
of the text file to human readers. 
 
This is an example text file that will be used for steganography testing. 
It contains multiple lines of text to demonstrate how data can be hidden 
in text files using different steganography techniques. 
 
The steganograph

### Extract Data from the Text

Now let's extract the hidden data from the text file.

In [7]:
# Extract the hidden data
extracted_data = text_stego.extract_data(output_text_path)

print(f"Extracted data size: {len(extracted_data)} bytes")

# Verify the extraction by comparing data lengths
if len(extracted_data) == len(secret_data):
    print("✅ Data size matches!")
else:
    print("❌ Data size doesn't match!")
    
# Verify the extraction by comparing the actual data
if extracted_data == secret_data:
    print("✅ Extraction successful! The extracted data matches the original.")
else:
    print("❌ Extraction failed! The data doesn't match.")

# Display the extracted image
try:
    from PIL import Image
    import io
    
    extracted_img = Image.open(io.BytesIO(extracted_data))
    plt.figure(figsize=(3, 3))
    plt.imshow(extracted_img)
    plt.title("Extracted Image from Text")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error displaying extracted image: {e}")

Extracted data size: 164 bytes
✅ Data size matches!
✅ Extraction successful! The extracted data matches the original.
Error displaying extracted image: name 'plt' is not defined


## 3. Using Huffman Compression

Let's demonstrate how to use Huffman coding to compress data before hiding it.

In [4]:
# Create some repetitive text data (good for compression)
repetitive_text = "This is a repetitive text. " * 50
print(f"Original text length: {len(repetitive_text)} characters")

# Encode the text using Huffman coding
encoded_data, codes = huffman_encode(repetitive_text)

# Convert the encoded bit string to bytes for storage
encoded_bytes = bits_to_bytes(encoded_data)
print(f"Compressed data length: {len(encoded_bytes)} bytes")

# Calculate compression ratio
original_bytes = len(repetitive_text.encode('utf-8'))
compression_ratio = original_bytes / len(encoded_bytes)
print(f"Compression ratio: {compression_ratio:.2f}x")

# We need to store the codes along with the compressed data
import pickle
compressed_package = {
    'data': encoded_bytes,
    'codes': codes
}
packaged_data = pickle.dumps(compressed_package)

# Hide this compressed package in an image
compressed_image_stego = LSBImageSteganography()
compressed_output_path = os.path.join(EXAMPLES_DIR, 'compressed_output_image.png')
compressed_image_stego.hide_data(image_path, packaged_data, compressed_output_path)

print(f"Compressed data hidden in image: {compressed_output_path}")

Original text length: 1350 characters
Compressed data length: 575 bytes
Compression ratio: 2.35x


NameError: name 'image_path' is not defined

### Extract and Decompress the Data

In [ ]:
# Extract the compressed package
extracted_package_data = compressed_image_stego.extract_data(compressed_output_path)
extracted_package = pickle.loads(extracted_package_data)

# Get the encoded data and codes
extracted_encoded_bytes = extracted_package['data']
extracted_codes = extracted_package['codes']

# Convert the bytes back to a bit string
extracted_encoded_data = bytes_to_bits(extracted_encoded_bytes)

# Decode using Huffman coding
decoded_text = huffman_decode(extracted_encoded_data, extracted_codes)

print(f"Decoded text length: {len(decoded_text)} characters")

# Verify the decompression was successful
if decoded_text == repetitive_text:
    print("✅ Decompression successful! The decoded text matches the original.")
else:
    print("❌ Decompression failed! The texts don't match.")

# Show a snippet of the decoded text
print("\nDecoded text snippet:")
print(decoded_text[:100] + "...")

## 4. Advanced: Zero-Width Text Steganography

Let's try the zero-width character method for text steganography, which is even more discreet than whitespace.

In [ ]:
# Create a secret message
secret_message = "This message is hidden using zero-width characters!"
secret_data = secret_message.encode('utf-8')

# Initialize the zero-width text steganography object
zw_stego = ZeroWidthTextSteganography()

# Hide the data in the text file
zw_output_path = os.path.join(EXAMPLES_DIR, 'zerowidth_output_text.txt')
result_path = zw_stego.hide_data(text_path, secret_data, zw_output_path)

print(f"Data hidden in text using zero-width characters: {result_path}")

# Display the output text
with open(zw_output_path, 'r', encoding='utf-8') as f:
    zw_stego_text = f.read()

print("\nText with Hidden Data (looks identical but contains invisible characters):")
print("---------------------")
print(zw_stego_text)
print("---------------------")

In [ ]:
# Extract the hidden data
extracted_zw_data = zw_stego.extract_data(zw_output_path)
extracted_zw_message = extracted_zw_data.decode('utf-8')

print(f"Extracted message: {extracted_zw_message}")

# Verify the extraction was successful
if extracted_zw_message == secret_message:
    print("✅ Extraction successful! The extracted message matches the original.")
else:
    print("❌ Extraction failed! The messages don't match.")

## Conclusion

In this notebook, we've demonstrated several steganography techniques:

1. LSB Image Steganography - Hiding data in the least significant bits of image pixels
2. Whitespace Text Steganography - Hiding data using spaces and tabs at the end of lines
3. Data Compression with Huffman Coding - Compressing data before hiding it
4. Zero-Width Text Steganography - Hiding data using invisible Unicode characters

These techniques allow you to hide information in plain sight, with the carrier files looking completely normal to the casual observer.